# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [1]:
import requests
import io
import pandas as pd
import numpy as np


Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

Solo así me dejó correr el código. También tuve que hacer pip install de pandas.

In [2]:
url = 'https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true'

# Descargar el archivo
response = requests.get(url)
response.raise_for_status()  # Verifica errores de conexión

# Cargar el contenido descargado directamente en pandas
loans = pd.read_csv(io.BytesIO(response.content), compression='gzip', encoding_errors='ignore')

print(loans.head())

/var/folders/t7/x59wmtvn5zz3dylxlv_fcch80000gn/T/ipykernel_23479/249270875.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv(io.BytesIO(response.content), compression='gzip', encoding_errors='ignore')


   Unnamed: 0        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv  \
0           0  38098114        NaN    15000.0      15000.0          15000.0   
1           1  36805548        NaN    10400.0      10400.0          10400.0   
2           2  37842129        NaN    21425.0      21425.0          21425.0   
3           3  37612354        NaN    12800.0      12800.0          12800.0   
4           4  37662224        NaN     7650.0       7650.0           7650.0   

         term  int_rate  installment grade  ...  \
0   60 months     12.39       336.64     C  ...   
1   36 months      6.99       321.08     A  ...   
2   60 months     15.59       516.36     D  ...   
3   60 months     17.14       319.08     D  ...   
4   36 months     13.66       260.20     C  ...   

  hardship_payoff_balance_amount hardship_last_payment_amount  \
0                            NaN                          NaN   
1                            NaN                          NaN   
2                      

## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [3]:
column_types =loans.dtypes
column_types

Unnamed: 0                 int64
id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 151, dtype: object

## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

Tuve que hacer pip instal openpyxl

In [4]:
url = 'https://resources.lendingclub.com/LCDataDictionary.xlsx'

# Descargar el archivo con requests
response = requests.get(url)
response.raise_for_status()  # Asegúrate de que la descarga fue exitosa

# Leer el archivo descargado con pandas
datos_dict = pd.read_excel(io.BytesIO(response.content), engine='openpyxl')
datos_dict.columns = ['feature', 'description']

print(datos_dict.head())

                feature                                        description
0        acc_now_delinq  The number of accounts on which the borrower i...
1  acc_open_past_24mths         Number of trades opened in past 24 months.
2            addr_state  The state provided by the borrower in the loan...
3              all_util              Balance to credit limit on all trades
4            annual_inc  The self-reported annual income provided by th...


In [5]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


Reorganizar las entradas para que estén en el mismo orden que el loans

In [6]:
# Assuming 'loans' is your DataFrame
loans_columns = loans.columns.tolist()  # Get the list of columns from the loans DataFrame

# Reorder datos_dict based on the columns in loans
# We will filter out features that are not in loans_columns
ordered_dict = datos_dict[datos_dict['feature'].isin(loans_columns)].copy()
ordered_dict = ordered_dict.set_index('feature').reindex(loans_columns).reset_index()

# Display the ordered table
print(ordered_dict)

                   feature                                        description
0               Unnamed: 0                                                NaN
1                       id      A unique LC assigned ID for the loan listing.
2                member_id   A unique LC assigned Id for the borrower member.
3                loan_amnt  The listed amount of the loan applied for by t...
4              funded_amnt  The total amount committed to that loan at tha...
..                     ...                                                ...
146      settlement_status  The status of the borrower’s settlement plan. ...
147        settlement_date  The date that the borrower agrees to the settl...
148      settlement_amount  The loan amount that the borrower has agreed t...
149  settlement_percentage  The settlement amount as a percentage of the p...
150        settlement_term  The number of months that the borrower will be...

[151 rows x 2 columns]


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [7]:
# Codigo guardar
datos_dict.to_pickle('ordered_dict.pkl')

In [8]:
# Codigo para cargar
datos_dict_loaded = pd.read_pickle('ordered_dict.pkl')

## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

Cambiamos los tipos de datos
category abarca menos memoria
Int64 permite NAN
dates para fechas

In [9]:
# Ajustar tipos de datos
loans = loans.astype({
    'member_id': 'Int64',  # Identificador con valores nulos
    'term': 'category',
    'grade': 'category',
    'sub_grade': 'category',
    'home_ownership': 'category',
    'verification_status': 'category',
    'loan_status': 'category',
    'pymnt_plan': 'category',
    'zip_code': 'object',
    'addr_state': 'category',
    'hardship_flag': 'bool',
    'debt_settlement_flag': 'bool'
})

# Convertir columnas de fechas
date_columns = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d', 
                'last_credit_pull_d', 'hardship_start_date', 'hardship_end_date', 
                'payment_plan_start_date', 'debt_settlement_flag_date', 'settlement_date']
for col in date_columns:
    loans[col] = pd.to_datetime(loans[col], errors='coerce')

# Mostrar los nuevos tipos de datos
column_types = loans.dtypes
print(column_types)


/var/folders/t7/x59wmtvn5zz3dylxlv_fcch80000gn/T/ipykernel_23479/2347453219.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  loans[col] = pd.to_datetime(loans[col], errors='coerce')
/var/folders/t7/x59wmtvn5zz3dylxlv_fcch80000gn/T/ipykernel_23479/2347453219.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  loans[col] = pd.to_datetime(loans[col], errors='coerce')
/var/folders/t7/x59wmtvn5zz3dylxlv_fcch80000gn/T/ipykernel_23479/2347453219.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  loans[col] = pd.to_datetime(loans[col], errors='coerce')
/var/folders/t7/x59wmt

Unnamed: 0                        int64
id                                int64
member_id                         Int64
loan_amnt                       float64
funded_amnt                     float64
                              ...      
settlement_status                object
settlement_date          datetime64[ns]
settlement_amount               float64
settlement_percentage           float64
settlement_term                 float64
Length: 151, dtype: object


In [10]:
# Manejo seguro de 'Unnamed: 0'
if 'Unnamed: 0' in loans.columns:
    loans = loans.drop(columns=['Unnamed: 0'])

# emp_title
loans['emp_title'] = loans['emp_title'].astype(str)

# emp_length
unique_titles = loans['emp_length'].unique()
length_mapping = {
    '10+ years': 10,
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9
}
loans['emp_length'] = loans['emp_length'].replace(length_mapping)
loans['emp_length'] = loans['emp_length'].astype('Int64')  # Permite valores NaN

# home_ownership
loans['home_ownership'] = loans['home_ownership'].replace('ANY', 'OTHER')
loans['home_ownership'] = loans['home_ownership'].astype('category')

# disbursement_method
loans['disbursement_method'] = loans['disbursement_method'].astype('category')

# debt_settlement_flag
loans['debt_settlement_flag'] = loans['debt_settlement_flag'].map({'Y': True, 'N': False})

# debt_settlement_flag_date
loans['debt_settlement_flag_date'] = pd.to_datetime(
    loans['debt_settlement_flag_date'], format='%b-%Y', errors='coerce')

# settlement_status
loans['settlement_status'] = loans['settlement_status'].astype('category')

# settlement_date
loans['settlement_date'] = pd.to_datetime(
    loans['settlement_date'], format='%b-%Y', errors='coerce')


/var/folders/t7/x59wmtvn5zz3dylxlv_fcch80000gn/T/ipykernel_23479/3476301222.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  loans['emp_length'] = loans['emp_length'].replace(length_mapping)
/var/folders/t7/x59wmtvn5zz3dylxlv_fcch80000gn/T/ipykernel_23479/3476301222.py:27: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  loans['home_ownership'] = loans['home_ownership'].replace('ANY', 'OTHER')


In [11]:
column_types =loans.dtypes
column_types

id                                int64
member_id                         Int64
loan_amnt                       float64
funded_amnt                     float64
funded_amnt_inv                 float64
                              ...      
settlement_status              category
settlement_date          datetime64[ns]
settlement_amount               float64
settlement_percentage           float64
settlement_term                 float64
Length: 150, dtype: object

## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [14]:
# Función para manejar imputación categórica
def imputar_categoricas_a_missing(loans, categorias_a_missing):
    print("Imputando categóricas con 'MISSING'...")
    for column in categorias_a_missing:
        if column in loans.columns:
            if pd.api.types.is_categorical_dtype(loans[column]):
                if 'MISSING' not in loans[column].cat.categories:
                    loans[column] = loans[column].cat.add_categories(['MISSING'])
            loans[column] = loans[column].fillna('MISSING')
            print(f"Columna {column} imputada con 'MISSING'.")
        else:
            print(f"Columna {column} no encontrada en el DataFrame.")

# Función para imputar con -1
def imputar_a_menos1(loans, amenos1):
    print("Imputando con -1...")
    for col in amenos1:
        if col in loans.columns:
            loans[col] = loans[col].fillna(-1)
            print(f"Columna {col} imputada con -1.")
        else:
            print(f"Columna {col} no encontrada en el DataFrame.")

# Función para imputar con medianas y condiciones
def imputar_mediana_condicional(loans, columnas_condicionales, condiciones):
    print("Imputando con mediana condicional...")
    for col, (condicion_col, valor_condicion) in columnas_condicionales.items():
        if col in loans.columns:
            mediana = loans[loans[col].notna()][col].median()
            loans[col] = loans.apply(
                lambda row: valor_condicion if row[condicion_col] == 0 
                else mediana if pd.isna(row[col]) 
                else row[col],
                axis=1
            )
            print(f"Columna {col} imputada con mediana condicional.")
        else:
            print(f"Columna {col} no encontrada en el DataFrame.")

# Función para imputar fechas con placeholder
def imputar_fechas_placeholder(loans, columnas_fecha, placeholder_date):
    print("Imputando fechas con placeholder...")
    for col in columnas_fecha:
        if col in loans.columns:
            loans[col] = loans[col].fillna(placeholder_date)
            print(f"Columna {col} imputada con fecha placeholder.")
        else:
            print(f"Columna {col} no encontrada en el DataFrame.")

# Función para imputar con 0
def imputar_a_cero(loans, columnas_a_cero):
    print("Imputando con 0...")
    for col in columnas_a_cero:
        if col in loans.columns:
            loans[col] = loans[col].fillna(0)
            print(f"Columna {col} imputada con 0.")
        else:
            print(f"Columna {col} no encontrada en el DataFrame.")

# Función principal
def imputacion_loans(loans):
    print("Iniciando imputación...")
    
    # Columnas categóricas a imputar con 'MISSING'
    categorias_a_missing = ['hardship_type', 'hardship_reason', 'hardship_status', 'hardship_loan_status', 'settlement_status']
    imputar_categoricas_a_missing(loans, categorias_a_missing)

    # Columnas a imputar con -1
    amenos1 = ['emp_length', 'mths_since_last_delinq', 'mths_since_last_record', 'mths_since_last_major_derog', 
               'mo_sin_old_il_acct', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 
               'mths_since_recent_inq', 'mths_since_recent_revol_delinq']
    imputar_a_menos1(loans, amenos1)

    # Columnas a imputar con medianas condicionales
    columnas_condicionales = {
        'revol_util': ('revol_bal', 0),
        'bc_util': ('bc_open_to_buy', 0)
    }
    imputar_mediana_condicional(loans, columnas_condicionales)

    # Columnas de fechas a imputar con un placeholder
    columnas_fecha = ['hardship_start_date', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 
                      'hardship_end_date', 'payment_plan_start_date', 'debt_settlement_flag_date', 
                      'settlement_date']
    imputar_fechas_placeholder(loans, columnas_fecha, pd.Timestamp('1900-01-01'))

    # Columnas a imputar con 0
    columnas_a_cero = ['bc_open_to_buy', 'num_tl_120dpd_2m', 'percent_bc_gt_75', 'deferral_term',
                        'hardship_amount', 'hardship_length', 'hardship_dpd', 
                        'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 
                        'hardship_last_payment_amount', 'settlement_amount', 'settlement_percentage', 
                        'settlement_term']
    imputar_a_cero(loans, columnas_a_cero)

    # Detectar columnas completamente nulas y eliminarlas
    columnas_completamente_nulas = [col for col in loans.columns if loans[col].isna().all()]
    if columnas_completamente_nulas:
        loans.drop(columns=columnas_completamente_nulas, inplace=True)
        print(f"Columnas eliminadas (completamente nulas): {columnas_completamente_nulas}")
    else:
        print("No hay columnas completamente nulas.")

    # Detectar columnas con valores faltantes restantes
    columnas_con_faltantes = [col for col in loans.columns if loans[col].isna().any()]
    if columnas_con_faltantes:
        print("Columnas con valores faltantes restantes:", columnas_con_faltantes)
    else:
        print("No hay columnas con valores faltantes.")

# Asegúrate de llamar a la función principal con un DataFrame
# Ejemplo:
# loans = pd.read_csv('ruta_a_tu_archivo.csv')
# imputacion_loans(loans)


### Codigo para salvar y cargar JSONs

In [15]:
import json
import pandas as pd

def create_imputation_json(loans, median_values):
    """
    Crea un archivo JSON con las estrategias de imputación aplicadas a las columnas del DataFrame.
    """
    # Diccionario para almacenar la información de imputación
    imputation_info = {}

    # Columnas eliminadas completamente nulas
    columnas_completamente_nulas = [col for col in loans.columns if loans[col].isna().all()]
    for column in columnas_completamente_nulas:
        imputation_info[column] = {
            'estrategia': 'Dropeamos columna - todos valores NaN',
            'valor': None
        }

    # Imputación con -1
    amenos1 = ['emp_length', 'mths_since_last_delinq', 'mths_since_last_record', 
               'mths_since_last_major_derog', 'mo_sin_old_il_acct', 'mths_since_recent_bc', 
               'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq']
    for col in amenos1:
        imputation_info[col] = {
            'estrategia': 'Indicar que no con identificador negativo, un 0 implicaria cosas',
            'valor': -1
        }

    # Imputación con 0
    columnas_a_cero = ['bc_open_to_buy', 'num_tl_120dpd_2m', 'percent_bc_gt_75', 'deferral_term',
                       'hardship_amount', 'hardship_length', 'hardship_dpd', 
                       'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 
                       'hardship_last_payment_amount', 'settlement_amount', 'settlement_percentage', 
                       'settlement_term']
    for col in columnas_a_cero:
        imputation_info[col] = {
            'estrategia': 'Identificador para decir que es nada',
            'valor': 0
        }

    # Fechas con placeholder
    columnas_fecha = ['hardship_start_date', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 
                      'hardship_end_date', 'payment_plan_start_date', 'debt_settlement_flag_date', 
                      'settlement_date']
    for col in columnas_fecha:
        imputation_info[col] = {
            'estrategia': 'Indicador que no hay fecha registrada',
            'valor': '1900-01-01'
        }

    # Imputación condicional con mediana
    columnas_condicionales = ['revol_util', 'bc_util']
    for i, col in enumerate(columnas_condicionales):
        imputation_info[col] = [
            {'estrategia': 'median imputation', 'valor': median_values[i]},
            {'estrategia': 'identificador', 'valor': 0}
        ]

    # Categóricas con 'MISSING'
    categorias_a_missing = ['hardship_type', 'hardship_reason', 'hardship_status', 
                             'hardship_loan_status', 'settlement_status']
    for col in categorias_a_missing:
        imputation_info[col] = {
            'estrategia': 'Identificador categorico',
            'valor': 'MISSING'
        }

    # Agregar columnas sin imputación explícita
    all_columns = loans.columns.tolist()
    for col in all_columns:
        if col not in imputation_info:
            imputation_info[col] = {
                'estrategia': 'no imputation',
                'valor': None
            }

    # Guardar el diccionario como JSON
    with open('imputation_strategies.json', 'w') as json_file:
        json.dump(imputation_info, json_file, indent=4)
    print("Archivo 'imputation_strategies.json' guardado correctamente.")

def load_and_print_json(file_path):
    """
    Carga y muestra un archivo JSON con las estrategias de imputación.
    """
    try:
        with open(file_path, 'r') as json_file:
            imputation_info = json.load(json_file)
        print(json.dumps(imputation_info, indent=4))  # Pretty print
    except FileNotFoundError:
        print(f"El archivo {file_path} no existe.")
    except json.JSONDecodeError:
        print(f"Error al decodificar el JSON en el archivo {file_path}.")

# Ejemplo de uso:
# Supongamos que loans es tu DataFrame y median_values contiene las medianas calculadas
# loans = pd.read_csv("tu_dataset.csv")
# median_values = [10.5, 20.0]  # Ejemplo de valores calculados
# create_imputation_json(loans, median_values)
# load_and_print_json('imputation_strategies.json')
